In [1]:
import os
import pysam
import remora
import sys
sys.path.append('..//')
import shannon_entropies as sp
import pod5
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
import line_profiler as lprof

In [2]:
def load_pod5_data(p5_path):
    
    # For some godforsaken reason, pod5 somehow consumes upwards of 6 GiB of memory when this runs, 
    # and it DOES NOT DISSAPEAR. ESLIT'rkjasvkajs;flksarejva;lkrvj
    data_list = []
    with pod5.Reader(p5_path) as pod5_file:
        for read in pod5_file.reads():
            seq_id = read.read_id
            signal = read.signal_pa
            freq = read.run_info.sample_rate
            data_dict = {'seq_id': str(seq_id),
                         'signal': signal,
                         'freq': freq}
            data_list.append(data_dict)
            
    return data_list

#https://pod5-file-format.readthedocs.io/en/latest/reference/api/pod5.reader.html#pod5.reader.ReadRecord

In [3]:
def split_signal(stride_length, raw_moves, signal):
    stride_moves = []
    for i in range(len(raw_moves)):
        if raw_moves[i]==0:
            for j in range(stride_length):
                stride_moves.append(raw_moves[i])
        else:
            stride_moves.append(1)
    stride_moves = np.array(stride_moves)
    stride_indicies = np.where(stride_moves == 1)[0]
    obs_signals = []
    for i in range(len(stride_indicies)-1):
        beg_index = stride_indicies[i]
        end_dex = stride_indicies[i+1]
        obs_signal = signal[beg_index:end_dex]
        obs_signals.append(obs_signal)
    
    return obs_signals

In [4]:
def align_observation_ops(cig, sig, ref):
    observation_dict_list = []
    observation_indexer = 0
    for observation in cig:
        ob_type = observation[0]
        ob_len = observation[1]
        for i in range(ob_len):
            try:
                observation_dict = {'operation':ob_type,
                                    'signal':sig[i+observation_indexer]}
            except:
                observation_dict = {'operation':ob_type,
                                    'signal':None}
            observation_dict_list.append(observation_dict)
        observation_indexer += ob_len
    obs_df = pd.DataFrame(observation_dict_list).shift(ref)
    valued_operations = obs_df[(obs_df['operation'] != 4) |( obs_df['operation'] != 4)].reset_index(drop='true')

    return valued_operations

In [40]:
def shift_to_alignment(ops, sigs, seq, quals, ref_len):
    base_position = 0
    base_keys = np.arange(ref_len).tolist()
    base_dict = {key:[] for key in base_keys}
    qual_dict =  {key:[] for key in base_keys}
    sig_dict = {key:[] for key in base_keys}
    for op in ops:
        if (op == str(0.0)):
            base_dict[base_position].append(seq[base_position])
            qual_dict[base_position].append(qual[base_position])
            sig_dict[base_position].append(sigs[base_position])
            base_position += 1
        elif (op == str(1.0)):
            base_dict[base_position].append("+")
            qual_dict[base_position].append(seq[base_position])
            sig_dict[base_position].append(sigs[base_position])
        elif(op == str(2.0)):
            seq.insert(base_position, "-")
            sigs.insert(base_position, "D")
            quals.insert(base_position, None)
            base_dict[base_position].append("-")
            qual_dict[base_position].append(None)
            sig_dict[base_position].append(None)
            base_position += 1
        elif(op == 'nan'):
            # Nan is shift
            seq.insert(base_position, "N")
            quals.insert(base_position, None)
            sigs.insert(base_position, "S")
            base_dict[base_position].append("")
            qual_dict[base_position].append(None)
            sig_dict[base_position].append(None)
            base_position += 1
            
    return [base_dict, qual_dict, sig_dict]

In [41]:
def merge_bam_reads_by_id(bam_list_dict, read_list_dict, consensus_list_dict):
        
    # this code can be made more time efficient and memory-chunking but that's not important rn
    bld = pd.DataFrame(bam_list_dict)
    rld = pd.DataFrame(read_list_dict)
    cld = pd.DataFrame(consensus_list_dict)
    
    merged_data_df = pd.merge(bld, rld, on='seq_id', how='left')
    
    for i in range(len(cld)):
        #merged_data_df.at[]
        indexes = merged_data_df[merged_data_df['ref_name'] == cld['id'][i]].index
        merged_data_df.loc[indexes,'ref_seq'] = cld['seq'][i]
    
    
    data_dict = merged_data_df.to_dict('records')

    return data_dict

In [7]:
# Load the paths
bam_path = "../../../data/large_working_directory/rough_consensus_output/large_align.bam"
pod5_path = "../../../data/large_working_directory/merged_pod5/merged.pod5"
ref_fasta = "../../../data/reads_large/230725_PZ_lib_v4_r10/fasta/good.fa"

# Use the shannon entropy methods to load the bam
loaded_bam = sp.load_in_data(bam_path)

print(sys.getsizeof(loaded_bam))

#Load the consensus reference fasta using shannon entropy methods
consens = sp.consensus_formatter(ref_fasta)

# Load the pod5 read data using load_pod5_data
dl = load_pod5_data(pod5_path)

# Merge the data
merged_list_dict = merge_bam_reads_by_id(loaded_bam, dl, consens)

1671784


In [ ]:
# Loop through this section for each read in the data, in other words: Turn into its own method that is then
# Applied to the merged_data_df, or even the merged_data_df thats merged with the consnensus fasta------------
# Extract the stride, raw moves, signal, of the first read
index = 1
stride = merged_data_df.iloc[index]['moves'][0]
raw = merged_data_df.iloc[index]['moves'][1:]
sig = merged_data_df.iloc[index]['signal']

# split the signal into discrete parts by the moves
split_sig = split_signal(stride, raw, sig)

# Get cigar string and reference position
cig_seq = merged_data_df['cigar'][index]
ref_pos = merged_data_df['ref'][index]

# Use the signal, cigar, and reference to get the valued operations and signals
val_ops = align_observation_ops(cig_seq, sig, ref_pos)

# get the operation series as a string:
operations = val_ops['operation'].apply(str)

# get the quality scores for the current read
qual = merged_data_df['quality'][index]

# get the signal as the list
signals = list(val_ops['signal'])

# get the read sequence as a list
seq_list = list(merged_data_df['sequence'][index])

#get the reference sequence id
ref_seq_id = merged_data_df['ref_name'][index]

# get the reference sequence length
ref_length = len(cons_df[cons_df['id'] == ref_seq_id]['seq'].values[0])

# Get the alignment shift:

alignment_shift = shift_to_alignment(operations, signals, seq_list, qual, ref_length)
# ------- This should probably use .apply(), but there may be memory issues.

In [36]:
profiler = lprof.LineProfiler()
def merge_bam_reads_by_id(bam_list_dict, read_list_dict, consensus_list_dict):
        
    read_ids = [str(x['seq_id']) for x in bam_list_dict][:10]
    
    merged_bam_reads = []
    for i in range(len(read_ids)):
        #print("{}/{}     ".format(i, len(read_ids)), end='\r')
        read = read_ids[i]
        bam_read_dict = list(d for d in bam_list_dict if d["seq_id"] == read)[0]
        raw_read_ls = list(d for d in read_list_dict if d["seq_id"] == read)
        if len(raw_read_ls) > 0:
            raw_read_dict = raw_read_ls[0]

            bam_read_dict['signal'] = raw_read_dict['signal']
            bam_read_dict['freq'] = raw_read_dict['freq']
            merged_bam_reads.append(bam_read_dict)
        
    

    return merged_bam_reads

profiled_func = profiler(merge_bam_reads_by_id)
profiled_func(loaded_bam, dl, consens) 
profiler.print_stats()

Timer unit: 1e-09 s

Total time: 0.639848 s
File: /tmp/ipykernel_29956/237270870.py
Function: merge_bam_reads_by_id at line 2

Line #      Hits         Time  Per Hit   % Time  Line Contents
     2                                           def merge_bam_reads_by_id(bam_list_dict, read_list_dict, consensus_list_dict):
     3                                               
     4         1   44547050.0    4e+07      7.0      read_ids = [str(x['seq_id']) for x in bam_list_dict][:10]
     5                                               
     6         1       1031.0   1031.0      0.0      merged_bam_reads = []
     7        11      11536.0   1048.7      0.0      for i in range(len(read_ids)):
     8                                                   #print("{}/{}     ".format(i, len(read_ids)), end='\r')
     9        10       7081.0    708.1      0.0          read = read_ids[i]
    10        10  374431873.0    4e+07     58.5          bam_read_dict = list(d for d in bam_list_dict if d["seq_id

In [ ]:
all_keys

In [8]:
sys.getsizeof(merged_data_df)

6868089114

In [ ]:
merged_data_df['reference_seq'] = merged_data_df['ref_name'].apply(lambda x: get_ref_by_id(x, cons_df))

In [17]:
merged_data_df['ref_name'][0]

'Pool=ACGTAACTTGGTTTGTTCCCTGAA-AGTCAGCT+CGAGGAGGTTCACTGGGTAGTAAG-TTCCAGGA+XPOS[P:66]'

In [18]:
merged_data_df['ref_name'][0]

False

In [22]:
merged_data_df['ref_name'][0] in cons_df['id']

False

In [ ]:
def get_alignment_info(, ):